# **Penting**
- Jangan mengubah atau menambahkan cell text yang sudah disediakan, Anda hanya perlu mengerjakan cell code yang sudah disediakan.
- Pastikan seluruh kriteria memiliki output yang sesuai, karena jika tidak ada output dianggap tidak selesai.
- Misal, Anda menggunakan df = df.dropna() silakan gunakan df.isnull().sum() sebagai tanda sudah berhasil. Silakan sesuaikan seluruh output dengan perintah yang sudah disediakan.
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Pastikan Anda menggunakan variabel df dari awal sampai akhir dan tidak diperbolehkan mengganti nama variabel tersebut.
- Hapus simbol pagar (#) pada kode yang bertipe komentar jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan
- Pastikan Anda mengerjakan sesuai section yang sudah diberikan tanpa mengubah judul atau header yang disediakan.

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [14]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV


# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [15]:
# Gunakan dataset hasil clustering yang memiliki fitur Target
# Silakan gunakan dataset data_clustering jika tidak menerapkan Interpretasi Hasil Clustering [Advanced]
# Silakan gunakan dataset data_clustering_inverse jika menerapkan Interpretasi Hasil Clustering [Advanced]
# Lengkapi kode berikut
df = pd.read_csv("data_clustering_inverse.csv")

In [16]:
# Tampilkan 5 baris pertama dengan function head.
df.head()

,TransactionAmount,TransactionDate,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,Target
0,$0-300 (Very Low),2023-04-11 16:29:14,Debit,San Diego,ATM,Oldest,Doctor,81.0,1.0,$6001-9000 (Medium),2024-11-04 08:08:08,1
1,$1201-1500 (High Activity),2023-06-27 16:44:19,Debit,Houston,ATM,Oldest,Doctor,141.0,1.0,$12001-15000 (Very High),2024-11-04 08:09:35,1
2,$601-900 (Medium-Low),2023-07-10 18:16:08,Debit,Mesa,Online,Young,Student,56.0,1.0,$0-3000 (Very Low),2024-11-04 08:07:04,3
3,$601-900 (Medium-Low),2023-05-05 16:32:11,Debit,Raleigh,Online,Young,Student,25.0,1.0,$12001-15000 (Very High),2024-11-04 08:09:06,3
4,$0-300 (Very Low),2023-10-16 17:51:24,Credit,Atlanta,Online,Adult,Student,198.0,1.0,$9001-12000 (High),2024-11-04 08:06:39,1


In [17]:
# Melihat informasi dari Dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2514 entries, 0 to 2513
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   TransactionAmount        2514 non-null   object 
 1   TransactionDate          2486 non-null   object 
 2   TransactionType          2484 non-null   object 
 3   Location                 2484 non-null   object 
 4   Channel                  2487 non-null   object 
 5   CustomerAge              2514 non-null   object 
 6   CustomerOccupation       2491 non-null   object 
 7   TransactionDuration      2514 non-null   float64
 8   LoginAttempts            2514 non-null   float64
 9   AccountBalance           2514 non-null   object 
 10  PreviousTransactionDate  2490 non-null   object 
 11  Target                   2514 non-null   int64  
dtypes: float64(2), int64(1), object(9)
memory usage: 235.8+ KB


In [18]:
numeric_features = ['TransactionDuration', 'LoginAttempts']
categorical_features = ['TransactionAmount', 'TransactionDate', 'TransactionType', 'Location', 'Channel', 'CustomerAge', 'CustomerOccupation','AccountBalance', 'PreviousTransactionDate']

# Custom LabelEncoder
def label_encode_columns(X):
    X = pd.DataFrame(X).copy()
    for col in X.columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
    return X
# Bungkus ke FunctionTransformer (stateless)
label_encoder_transformer = FunctionTransformer(label_encode_columns, validate=False)


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('labelenc', label_encoder_transformer)
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [19]:
# Menggunakan train_test_split() untuk melakukan pembagian dataset.
X = df.drop('Target', axis=1)
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [20]:
# Buatlah model klasifikasi menggunakan Decision Tree
model_dc = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))])

# fit model
model_dc.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['TransactionDuration',
                                                   'LoginAttempts']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('labelenc',
                                                                   FunctionTransformer(func=<function label_encode_columns at 0x7d59af300c20>))]),
                                                  ['TransactionAmount',
                                                   'TransactionDate',
                                                   'TransactionType',
                                                   'Location', 'Channel',
                                                   'CustomerAge',
                                                   'CustomerOccupation',
                                                   'AccountBalance',
                                                   'PreviousTransactionDate'])])),
                ('classifier', DecisionTreeClassifier(random_state=42))])

In [21]:
# Menyimpan Model
joblib.dump(model_dc, 'decision_tree_model.h5')

['decision_tree_model.h5']

# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [22]:
# Melatih model menggunakan algoritma klasifikasi scikit-learn selain Decision Tree.
model_rf = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))])

# fit model
model_rf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['TransactionDuration',
                                                   'LoginAttempts']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('labelenc',
                                                                   FunctionTransformer(func=<function label_encode_columns at 0x7d59af300c20>))]),
                                                  ['TransactionAmount',
                                                   'TransactionDate',
                                                   'TransactionType',
                                                   'Location', 'Channel',
                                                   'CustomerAge',
                                                   'CustomerOccupation',
                                                   'AccountBalance',
                                                   'PreviousTransactionDate'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [23]:
# Melatih model menggunakan algoritma klasifikasi scikit-learn selain Decision Tree.
model_gbc = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(random_state=42))])

# fit model
model_gbc.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['TransactionDuration',
                                                   'LoginAttempts']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('labelenc',
                                                                   FunctionTransformer(func=<function label_encode_columns at 0x7d59af300c20>))]),
                                                  ['TransactionAmount',
                                                   'TransactionDate',
                                                   'TransactionType',
                                                   'Location', 'Channel',
                                                   'CustomerAge',
                                                   'CustomerOccupation',
                                                   'AccountBalance',
                                                   'PreviousTransactionDate'])])),
                ('classifier', GradientBoostingClassifier(random_state=42))])

In [24]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada seluruh algoritma yang sudah dibuat.
# Fungsi untuk mengevaluasi dan mengembalikan hasil sebagai kamus
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-Score': f1}

# Mengevaluasi setiap model dan mengumpulkan hasilny
results = {
    'Decision Tree': evaluate_model(model_dc, X_test, y_test),
    'Random Forest': evaluate_model(model_rf, X_test, y_test),
    'Gradient Boosting': evaluate_model(model_gbc, X_test, y_test)
}

# Buat DataFrame dengan hasil
rows = []
for model_name, metrics in results.items():
    rows.append({
        'Model': model_name,
        'Accuracy (Test)': metrics['Accuracy'],
        'Precision': metrics['Precision'],
        'Recall': metrics['Recall'],
        'F1-Score': metrics['F1-Score']
    })

# Konversi daftar kamus ke DataFrame
summary_df = pd.DataFrame(rows)

# Tampilkan DataFrame
print(summary_df)

               Model  Accuracy (Test)  Precision    Recall  F1-Score
0      Decision Tree         0.976143   0.976226  0.976143  0.976137
1      Random Forest         0.986083   0.986120  0.986083  0.986018
2  Gradient Boosting         0.988072   0.988151  0.988072  0.988045


In [25]:
# Menyimpan Model Selain Decision Tree
# Model ini bisa lebih dari satu
# import joblib
joblib.dump(model_rf, 'explore_RandomForest_classification.h5')

['explore_RandomForest_classification.h5']

Hyperparameter Tuning Model

Pilih salah satu algoritma yang ingin Anda tuning

In [26]:
# Lakukan Hyperparameter Tuning dan Latih ulang.
# Lakukan dalam satu cell ini saja.
model_dc_tuning = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

param_grid = {
    "classifier__criterion": ["gini", "entropy", "log_loss"],
    "classifier__splitter": ["best", "random"],
    "classifier__max_depth": [None, 3, 5, 7, 9, 12, 15, 20],
    "classifier__min_samples_split": [2, 5, 10, 20, 50],
    "classifier__min_samples_leaf": [1, 2, 5, 10, 20],
    "classifier__max_features": [None, "sqrt", "log2", 0.5, 0.8],
    "classifier__class_weight": [None, "balanced"],
    "classifier__ccp_alpha": [0.0, 1e-4, 1e-3, 1e-2],
    }

grid_dc = GridSearchCV(model_dc_tuning, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_dc.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['TransactionDuration',
                                                                          'LoginAttempts']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('labelenc',
                                                                                          FunctionTransformer(func=<functi...
                         'classifier__class_weight': [None, 'balanced'],
                         'classifier__criterion': ['gini', 'entropy',
                                                   'log_loss'],
                         'classifier__max_depth': [None, 3, 5, 7, 9, 12, 15,
                                                   20],
                         'classifier__max_features': [None, 'sqrt', 'log2', 0.5,
                                                      0.8],
                         'classifier__min_samples_leaf': [1, 2, 5, 10, 20],
                         'classifier__min_samples_split': [2, 5, 10, 20, 50],
                         'classifier__splitter': ['best', 'random']},
             scoring='accuracy')

In [27]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada algoritma yang sudah dituning.
# Fungsi untuk mengevaluasi dan mengembalikan hasil sebagai kamus
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-Score': f1}

# Mengevaluasi setiap model dan mengumpulkan hasilny
results = {
    'Decision Tree': evaluate_model(model_dc, X_test, y_test),
    'Random Forest': evaluate_model(model_rf, X_test, y_test),
    'Gradient Boosting': evaluate_model(model_gbc, X_test, y_test),
    'Decision Tree Tuning': evaluate_model(grid_dc, X_test, y_test)
}

# Buat DataFrame dengan hasil
rows = []
for model_name, metrics in results.items():
    rows.append({
        'Model': model_name,
        'Accuracy (Test)': metrics['Accuracy'],
        'Precision': metrics['Precision'],
        'Recall': metrics['Recall'],
        'F1-Score': metrics['F1-Score']
    })

# Konversi daftar kamus ke DataFrame
summary_df = pd.DataFrame(rows)

# Tampilkan DataFrame
print(summary_df)

                  Model  Accuracy (Test)  Precision    Recall  F1-Score
0         Decision Tree         0.976143   0.976226  0.976143  0.976137
1         Random Forest         0.986083   0.986120  0.986083  0.986018
2     Gradient Boosting         0.988072   0.988151  0.988072  0.988045
3  Decision Tree Tuning         0.986083   0.986123  0.986083  0.986052


In [30]:
# Menyimpan Model hasil tuning
# import joblib
joblib.dump(grid_dc, 'tuning_classification.h5')

['tuning_classification.h5']

End of Code